In [ ]:
import copclib as copc
import numpy as np
import laspy 
import torch
from torch_geometric.data import Dataset, Data
import os
import sys
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)
from torch_points3d.core.data_transform import GridSampling3D

In [ ]:
reader = copc.FileReader("/media/nvme/pcdata/tyrol/copc/splits_0/octree.copc.laz")
max_depth = reader.GetDepthAtResolution(0.5)
hierarchy = {str(node.key): node for node in reader.GetAllChildren() if node.key.d <= max_depth}

In [ ]:
header = reader.GetLasHeader()
start_key = copc.VoxelKey(10, 931, 517, 58)
nearest_depth, x, y = start_key.d, start_key.x, start_key.y
sample_bounds = copc.Box(copc.VoxelKey(nearest_depth, x, y, 0), header)
# Make the tile 2D
sample_bounds.z_min = sys.float_info.min
sample_bounds.z_max = sys.float_info.max


In [ ]:
sample_bounds

In [ ]:
# given a key, recursively check if each of its 8 children exist in the hierarchy
def get_all_key_children(key, max_depth, hierarchy, children):
    # stop once we reach max_depth, since none of its children can exist
    if key.d >= max_depth:
        return
    for child in key.GetChildren():
        if str(child) in hierarchy:
            print(str(child))
            print(str(hierarchy[str(child)].key))
            # if the key exists, add it to the output, and check if any of its children exist
            children.append(hierarchy[str(child)])
            get_all_key_children(child, max_depth, hierarchy, children)


In [ ]:
valid_nodes = {}
# check each possible z to see if it, or any of its parents, exist
for i, z in enumerate([56, 57, 58, 59]):
    start_key = copc.VoxelKey(nearest_depth, x, y, z)

    # start by checking if the node itself exists
    if str(start_key) in hierarchy:
        # if the node exists, then get all its children
        child_nodes = []
        get_all_key_children(start_key, max_depth, hierarchy, child_nodes)
        for child_node in child_nodes:
            valid_nodes[str(child_node.key)] = child_node
        #print(len(child_nodes))
    else:
        # if the node doens't exist, get its first parent to exist
        #print("HERE")
        while str(start_key) not in hierarchy:
            if start_key.d < 0:
                raise RuntimeError("This shouldn't happen!")
            start_key = start_key.GetParent()

    #print(len(valid_nodes))
    # then, get all nodes from depth 0 to the current depth
    key = start_key
    while key.IsValid():
        valid_nodes[str(key)] = hierarchy[str(key)]
        key = key.GetParent()
#print(len(valid_nodes))
# Process keys that exist
copc_points = copc.Points(header)

for node in valid_nodes.values():
    key = node.key
    copc_points.AddPoints(reader.GetPoints(node).GetWithin(sample_bounds))


In [ ]:
points = np.stack([copc_points.X, copc_points.Y, copc_points.Z], axis=1)
classification = np.array(copc_points.Classification).astype(np.int)
data = Data(pos=torch.from_numpy(points).type(torch.float), y=y)
data = GridSampling3D(0.5, True, "last")(data)
data

In [ ]:
print(data.coords.min(0))
print(data.coords.max(0))


In [ ]:
head = laspy.header.Header()
las = laspy.file.File("tile4.laz", mode="w", header=head)
las.header.scale = [header.scale.x, header.scale.y, header.scale.z]
las.header.offset = [header.offset.x, header.offset.y, header.offset.z]
las.x = np.array(data.pos[:,0])
las.y = np.array(data.pos[:,1])
las.z = np.array(data.pos[:,2])
las.close()